In [7]:
!pip uninstall graphviz --yes

Found existing installation: graphviz 0.20.3
Uninstalling graphviz-0.20.3:
  Successfully uninstalled graphviz-0.20.3


In [8]:
!pip install graphviz


  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from graphviz import Digraph

def thompson_construction(re_string):
    """
    Generates a Thompson NFA for the given regular expression, implementing '+' as 'RE.RE*'.
    """
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')  # Left-to-right visualization

    # States counter
    state_count = 0

    def new_state():
        nonlocal state_count
        state = f'q{state_count}'
        state_count += 1
        return state

    # Start with initial and final states
    start_state = new_state()
    final_state = new_state()

    # Stack for automata
    stack = []

    for char in re_string:
        if char.isalpha():  # Add a transition for letters
            s1 = new_state()
            s2 = new_state()
            dot.node(s1, s1, shape='circle')
            dot.node(s2, s2, shape='circle')
            dot.edge(s1, s2, label=char)
            stack.append((s1, s2))
        elif char == '|':  # Union
            a2, a1 = stack.pop(), stack.pop()
            s1 = new_state()
            s2 = new_state()
            dot.node(s1, s1, shape='circle')
            dot.node(s2, s2, shape='circle')
            dot.edge(s1, a1[0], label='ε')
            dot.edge(s1, a2[0], label='ε')
            dot.edge(a1[1], s2, label='ε')
            dot.edge(a2[1], s2, label='ε')
            stack.append((s1, s2))
        elif char == '*':  # Kleene Star
            a = stack.pop()
            s1 = new_state()
            s2 = new_state()
            dot.node(s1, s1, shape='circle')
            dot.node(s2, s2, shape='circle')
            dot.edge(s1, a[0], label='ε')
            dot.edge(s1, s2, label='ε')
            dot.edge(a[1], a[0], label='ε')
            dot.edge(a[1], s2, label='ε')
            stack.append((s1, s2))
        elif char == '+':  # One or More Repetition
            a = stack.pop()
            s1 = new_state()
            s2 = new_state()
            dot.node(s1, s1, shape='circle')
            dot.node(s2, s2, shape='circle')
            
            # Implementing RE+ as RE.RE*
            dot.edge(s1, a[0], label='ε')  # First occurrence of RE
            dot.edge(a[1], a[0], label='ε')  # Loop for RE*
            dot.edge(a[1], s2, label='ε')  # Transition to the final state
            stack.append((s1, s2))

    # Connect the resulting NFA to the start and final states
    nfa_start, nfa_end = stack.pop()
    dot.edge(start_state, nfa_start, label='ε')
    dot.edge(nfa_end, final_state, label='ε')
    dot.node(start_state, start_state, shape='circle', color='red')  # Start state
    dot.node(final_state, final_state, shape='doublecircle', color='green')  # Final state

    return dot

# Main function
def main():
    re_string = "(aba|ba|aaa)+"

    # Generate Thompson NFA
    print("Generating Thompson NFA...")
    thompson_nfa = thompson_construction(re_string)
    thompson_nfa.render('thompson_nfa', cleanup=True)
    print("Thompson NFA saved as 'thompson_nfa.png'.")

if __name__ == "__main__":
    main()


Generating Thompson NFA...
Thompson NFA saved as 'thompson_nfa.png'.


In [17]:
import re

In [18]:
def glushkov_construction(re_string):
    """
    Generates a Glushkov NFA for the given regular expression.
    """
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')  # Left-to-right visualization

    # Assign unique positions to each symbol in the regular expression
    positions = {m.start(): m.group() for m in re.finditer(r'[a-zA-Z]', re_string)}
    start_state = 'q0'
    dot.node(start_state, start_state, shape='circle', color='red')  # Start state

    # Add transitions based on the sequence of symbols
    prev_state = start_state
    for pos, char in positions.items():
        next_state = f'q{pos + 1}'
        dot.node(next_state, next_state, shape='circle')
        dot.edge(prev_state, next_state, label=char)
        prev_state = next_state

    # Add a final state
    final_state = f'q{len(positions) + 1}'
    dot.node(final_state, final_state, shape='doublecircle', color='green')  # Final state
    dot.edge(prev_state, final_state, label='ε')  # Transition to final state

    return dot


In [ ]:
import re
import random

# Regular expressions
regex1 = r"(aba|ba|aaa)+"
regex2 = r"(ab|abc|aba)+ba(c+)(cbb|bba|a)+"

In [40]:


# Generate sequences that belong to the given regular expressions
def generate_sequences(regex, num_sequences=10, min_length=3, max_length=60):
    if regex == regex1:
        base_patterns = ["aba", "ba", "aaa"]
    elif regex == regex2:
        base_patterns = ["ab", "abc", "aba"]
        tail_patterns = ["cbb", "bba", "a"]
    
    sequences = []
    for _ in range(num_sequences):
        length = random.randint(min_length, max_length)
        sequence = ""
        while len(sequence) < length:
            if regex == regex1:
                sequence += random.choice(base_patterns)
            elif regex == regex2:
                sequence += random.choice(base_patterns)  # First part
                sequence += "ba"  # Fixed 'ba'
                sequence += "c" * random.randint(1, 5)  # c+
                sequence += random.choice(tail_patterns)  # Tail patterns
        sequences.append(sequence[:length])
    return sequences

# Generate random sequences and inject examples
def generate_random_file(filename, alphabet="abc", num_sequences=1000, sequence_length=50, injected_sequences=[]):
    with open(filename, "w") as file:
        for _ in range(num_sequences - len(injected_sequences)):
            sequence = "".join(random.choice(alphabet) for _ in range(sequence_length))
            file.write(sequence + "\n")
        for seq in injected_sequences:
            file.write(seq + "\n")

# Match regular expressions against the file
def match_nfa_against_file(regex, filename):
    pattern = re.compile(regex)
    matches = []
    with open(filename, "r") as file:
        for line in file:
            sequence = line.strip()
            if pattern.fullmatch(sequence):
                matches.append(sequence)
    return matches

# Estimate frequency of matches
def estimate_frequency(num_matches, total_sequences):
    return num_matches / total_sequences


Generated sequences for Regex 1:
['aaabaaaaabaabaaaaaaa', 'aaaaaababaabaaaaaa', 'abaaaaabababaaaaaaaababaaba', 'ababaaaabaabaaaaaaabaabaaa', 'aaaababaabaaaabaabababaaaaabaaaabaabaaaaa']
Generated sequences for Regex 2:
['abcbaccccbbababaccbbaabcbacccccbba', 'abbacccbbaabcbacccaabbacccaababacccccbbabab', 'abbaccccccbbabc', 'ababacccccbbabab', 'abbaccccbbaabbaccccaabbaccccaabcbaccccaababa']
Random file 'random_sequences.txt' created with injected sequences.

Matches for Regex 1: 2
Matches for Regex 2: 0

Examples of matches for Regex 1:
['aaabaaaaabaabaaaaaaa', 'abaaaaabababaaaaaaaababaaba']

Examples of matches for Regex 2:
[]

Estimated frequency for Regex 1: 0.0020
Estimated frequency for Regex 2: 0.0000


In [ ]:
def nfa_to_dfa(nfa_transitions, start_state, accept_states, alphabet):
    """
    Converts an NFA to a DFA using subset construction.

    Parameters:
        nfa_transitions: Dict of NFA transitions {state: {symbol: [next_states]}}
        start_state: Initial state of the NFA
        accept_states: Set of accepting states in the NFA
        alphabet: List of input symbols

    Returns:
        A tuple (dfa_transitions, dfa_start_state, dfa_accept_states)
    """
    # Helper function: Compute epsilon-closure of a set of states
    def epsilon_closure(states, nfa_transitions):
        stack = list(states)
        closure = set(states)

        while stack:
            state = stack.pop()
            if state in nfa_transitions and 'ε' in nfa_transitions[state]:
                for next_state in nfa_transitions[state]['ε']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    # DFA structures
    dfa_transitions = {}
    dfa_start_state = frozenset(epsilon_closure([start_state], nfa_transitions))
    dfa_states = {dfa_start_state}
    unmarked_states = [dfa_start_state]
    dfa_accept_states = set()

    while unmarked_states:
        current_dfa_state = unmarked_states.pop()
        dfa_transitions[current_dfa_state] = {}

        # Check if this DFA state is an accept state
        if any(state in accept_states for state in current_dfa_state):
            dfa_accept_states.add(current_dfa_state)

        # Compute transitions for each symbol in the alphabet
        for symbol in alphabet:
            next_states = set()
            for nfa_state in current_dfa_state:
                if nfa_state in nfa_transitions and symbol in nfa_transitions[nfa_state]:
                    next_states.update(nfa_transitions[nfa_state][symbol])

            # Compute epsilon-closure of the reachable states
            next_closure = frozenset(epsilon_closure(next_states, nfa_transitions))
            if next_closure:
                dfa_transitions[current_dfa_state][symbol] = next_closure
                if next_closure not in dfa_states:
                    dfa_states.add(next_closure)
                    unmarked_states.append(next_closure)

    return dfa_transitions, dfa_start_state, dfa_accept_states


In [ ]:
def minimize_dfa(dfa_transitions, dfa_start_state, dfa_accept_states, alphabet):
    """
    Minimizes a DFA.

    Parameters:
        dfa_transitions: Dict of DFA transitions {state: {symbol: next_state}}
        dfa_start_state: Initial state of the DFA
        dfa_accept_states: Set of accepting states in the DFA
        alphabet: List of input symbols

    Returns:
        A tuple (min_dfa_transitions, min_dfa_start_state, min_dfa_accept_states)
    """
    # Partition states into accepting and non-accepting sets
    all_states = set(dfa_transitions.keys())
    partition = [dfa_accept_states, all_states - dfa_accept_states]

    # Refinement of partitions
    while True:
        new_partition = []
        for group in partition:
            # Split group based on transitions
            split = defaultdict(set)
            for state in group:
                transitions = tuple(
                    next((i for i, part in enumerate(partition) if dfa_transitions.get(state, {}).get(symbol) in part), None)
                    for symbol in alphabet
                )
                split[transitions].add(state)
            new_partition.extend(split.values())

        if new_partition == partition:
            break
        partition = new_partition

    # Map old states to new states
    state_map = {state: i for i, group in enumerate(partition) for state in group}

    # Build the minimized DFA
    min_dfa_transitions = {}
    min_dfa_accept_states = set()
    min_dfa_start_state = state_map[dfa_start_state]

    for state, transitions in dfa_transitions.items():
        new_state = state_map[state]
        if new_state not in min_dfa_transitions:
            min_dfa_transitions[new_state] = {}
        for symbol, next_state in transitions.items():
            min_dfa_transitions[new_state][symbol] = state_map[next_state]

    for group in partition:
        if group & dfa_accept_states:
            min_dfa_accept_states.add(state_map[next(iter(group))])

    return min_dfa_transitions, min_dfa_start_state, min_dfa_accept_states


In [ ]:
# Main program
def main():
    # Generate sequences for each regex
    sequences1 = generate_sequences(regex1, num_sequences=5, min_length=3, max_length=50)
    sequences2 = generate_sequences(regex2, num_sequences=5, min_length=10, max_length=60)
    injected_sequences = sequences1 + sequences2
    
    print("Generated sequences for Regex 1:")
    print(sequences1)
    print("Generated sequences for Regex 2:")
    print(sequences2)
    
    # Generate a random file and inject sequences
    random_filename = "random_sequences.txt"
    generate_random_file(random_filename, injected_sequences=injected_sequences)
    print(f"Random file '{random_filename}' created with injected sequences.")
    
    # Match regex 1 and regex 2 against the file
    matches1 = match_nfa_against_file(regex1, random_filename)
    matches2 = match_nfa_against_file(regex2, random_filename)
    
    print(f"\nMatches for Regex 1: {len(matches1)}")
    print(f"Matches for Regex 2: {len(matches2)}")
    
    # Display some matches
    print("\nExamples of matches for Regex 1:")
    print(matches1[:5])
    print("\nExamples of matches for Regex 2:")
    print(matches2[:5])
    
    # Estimate frequency of matches
    total_sequences = 1000
    frequency1 = estimate_frequency(len(matches1), total_sequences)
    frequency2 = estimate_frequency(len(matches2), total_sequences)
    print(f"\nEstimated frequency for Regex 1: {frequency1:.4f}")
    print(f"Estimated frequency for Regex 2: {frequency2:.4f}")

if __name__ == "__main__":
    main()